<a href="https://colab.research.google.com/github/tolewicz/Product_Reviews_Big_Data/blob/main/BIg_Data_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

--2020-05-05 08:40:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-05-05 08:40:23 (8.37 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets user_data.csv
#IT WORKED !! I CAN'T BELIVE IT WORKED !
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)


# Show DataFrame
user_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:
print(f'number of rows = {user_data_df.count()}')
print(f'number of columns {len(user_data_df.columns)}')

number of rows = 1785997
number of columns 15


In [ ]:
#Dropping nan
data_clean_na=user_data_df.dropna()
print(f'number of rows data na cleanned = {data_clean_na.count()}')

#Dropping the same rows
data_clean_double = data_clean_na.distinct()
print(f'number of rows data wo 2x rows = {data_clean_double.count()}')

#Dropping the data of same customer reviewing same product 2x
data_clean=data_clean_double.dropDuplicates(["customer_id","product_id"])
print(f'number of rows data wo 2x rows = {data_clean.count()}')





number of rows data na cleanned = 1785886
number of rows data wo 2x rows = 1785886
number of rows data wo 2x rows = 1779349


In [ ]:
#format daytime to date
from pyspark.sql.types import DateType
data_clean=data_clean.withColumn("review_date",data_clean["review_date"].cast(DateType()))
#data_na_clean.show()

In [ ]:
#verified purcahse
from pyspark.sql.functions import col
data_purchased = data_clean.filter(col("verified_purchase") == "Y")
#data_purchased.show()

In [ ]:
print(f'number of rows w. purchase verified = {data_purchased.count()}')


number of rows w. purchase verified = 1169400


In [ ]:
#checking existence of vine
vine_cleaned = data_clean.filter(col("vine") == "Y")

print(f'number of rows w. vine only from "na" cleaned data = {vine_cleaned.count()}')


number of rows w. vine only from "na" cleaned data = 4290


In [ ]:
review_id_table = data_clean.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()
review_id_table.dtypes

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3U67AEPQEY1HS|      11664|B00EFFW0HC|     277685961| 2014-11-02|
|R2KKNB3ZJW646C|      12443|B0082SICGQ|     789058612| 2012-12-18|
|R14LSVC3NE43IA|      14298|B004HX1QFY|     505269054| 2014-09-21|
|R30TIKP9YQEP6D|      14653|B00U6Y7ISC|     265723335| 2015-06-06|
|R3NK2IML8A31AB|      16964|B002BS47JE|     449719506| 2013-01-30|
|R3UU9WCPVW1QEB|      17289|B00E20STCK|     496418464| 2015-07-25|
|R2J58J8HJ7VFHI|      22150|B00EP5UODE|     259088061| 2015-07-19|
| RITMWPU9WN7AS|      36322|B007H45AZ2|     344618424| 2013-01-03|
|R2XA7S79IU8U7V|      41216|B0053OLY9O|     644540693| 2013-12-02|
|R3DCR966DTLVC2|      60986|B000VD9RKU|     493776828| 2013-02-06|
|R36ZWEW6Q9CUJP|      61533|B005TD4Q08|     660576667| 2013-11-10|
|R37NGPHVC8Q2FA|      64750|B00OGPTED4|     513232670| 2015-07

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [ ]:
products = data_clean.select(["product_id", "product_title"])
products.show()
#products.dtypes

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EFFW0HC|       Battlefield 4|
|B0082SICGQ|Xbox 360 250GB Ra...|
|B004HX1QFY|Nyko Intercooler ...|
|B00U6Y7ISC|Mario - Gold amii...|
|B002BS47JE|        Wii Fit Plus|
|B00E20STCK|SpongeBob Squarep...|
|B00EP5UODE|Yu-gi-oh Zexal Du...|
|B007H45AZ2|     Dead or Alive 5|
|B0053OLY9O|Wireless Stereo H...|
|B000VD9RKU|Wii GameBlaster L...|
|B005TD4Q08|NCAA Football 200...|
|B00OGPTED4|Sony Move Chargin...|
|B007MJSZDQ|Matney 2 Pack Xbo...|
|B00NQT48F0|Skylanders Mini A...|
|B00OFQFYVU|G941 Virtual 7.1 ...|
|B00DBDPOZ4|Xbox One Play and...|
|B004EDJOYC|Sniper: Ghost War...|
|B002I0J5NI|Super Street Figh...|
|B00000K4CR|           GameShark|
|B009M72E5Q|Nintendo Wii Cons...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
customers = data_clean.select(col('customer_id'),col('star_rating').alias("customer_count"))
customers.show()
customers.dtypes
# no customer_count column in main table

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51813110|             5|
|   26113368|             5|
|   47001176|             5|
|   25167068|             5|
|   40235148|             4|
|   30640147|             4|
|   10694970|             3|
|   35671670|             4|
|   45070066|             3|
|   25213553|             5|
|   45654033|             5|
|   23904949|             4|
|   48150210|             5|
|   27241271|             5|
|   47424514|             5|
|   44950283|             5|
|   40913597|             1|
|   26361763|             5|
|   17570349|             5|
|   28196210|             4|
+-----------+--------------+
only showing top 20 rows



[('customer_id', 'int'), ('customer_count', 'int')]

In [ ]:
vine_table = data_clean.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_table.show()
vine_table.dtypes

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3U67AEPQEY1HS|          5|            0|          1|   N|
|R2KKNB3ZJW646C|          5|            0|          0|   N|
|R14LSVC3NE43IA|          5|            0|          0|   N|
|R30TIKP9YQEP6D|          5|            0|          1|   N|
|R3NK2IML8A31AB|          5|            0|          0|   N|
|R3UU9WCPVW1QEB|          5|            0|          0|   N|
|R2J58J8HJ7VFHI|          5|            1|          1|   N|
| RITMWPU9WN7AS|          5|            0|          0|   N|
|R2XA7S79IU8U7V|          1|            1|          4|   N|
|R3DCR966DTLVC2|          3|            0|          0|   N|
|R36ZWEW6Q9CUJP|          5|            2|          2|   N|
|R37NGPHVC8Q2FA|          5|            0|          0|   N|
|R1EZO7EW7L1ZZC|          1|            0|          0|   N|
|R2U4V3Y3LKDZ33|          5|            

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [ ]:
# Configure settings for RDS 
mode = "append"
jdbc_url="jdbc:postgresql://challenge.cugxdv9d3wwt.us-west-1.rds.amazonaws.com:5432/data_hw_two"
config = {"user":"postgres", 
          "password": "xxxxxxx", 
          "driver":"org.postgresql.Driver"}



In [ ]:
# # Write dataframe to billing_info table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

# # Write dataframe to billing_info table in RDS
# customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

# # # Write dataframe to payment_info table in RDS
#products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

# # # Write DataFrame to active_user table in RDS
# vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)